In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.style.use('default')

sns.set(style='whitegrid')

# Análisis de frecuencia de eventos por hora, por día de la semana

En este notebook vamos a analizar la frecuencia que tienen ciertos eventos dependiendo del momento del día en el que se registren. 

En primer lugar consideraremos aquellos que estén relacionados a **cómo un usuario accede al sitio**, y por otro lado aquellos eventos referidos al **flujo de compra** de un producto.

Por lo tanto vamos a iniciar este análisis posando las siguientes preguntas:

1. ¿Se puede observar mayor actividad en ciertos días?

2. Además del día, ¿Existe una franja horaria en donde se registre una mayor cantidad de eventos? ¿Se mantiene el comportamiento entre los distintos eventos o varía dependiendo del tipo?

In [ ]:
df = pd.read_csv('data/events.csv', low_memory=False)

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['weekday'] = df['timestamp'].dt.weekday_name
df['hour'] = df['timestamp'].dt.hour

## Cantidad de eventos por hora y por día de la semana
Primero revisaremos el tráfico del sitio, sin separar por evento, para darnos una idea general de cómo está repartido la totalidad del tráfico según la franja horaria y día de la semana.

In [ ]:
eventos_df = df.loc[:,['timestamp', 'weekday', 'hour', 'event']]
eventos_df['weekday'] = pd.Categorical(eventos_df['weekday'],\
               categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'], ordered=True)

In [ ]:
# Creamos una tabla pivot para poder mostrar los datos que nos interesan utilizando un Heatmap.
_heatmap = eventos_df.pivot_table(index='weekday', columns='hour', values='event', aggfunc='count')
_heatmap

In [ ]:
def crear_grafico_heatmap(heatmap, color, titulo):
    g = sns.heatmap(heatmap, cmap=color)
    g.set_title(titulo, fontsize=16)
    g.set_xlabel("Hora", fontsize=14)
    g.set_ylabel("Día", fontsize=14)
    plt.xticks(rotation=0)
    plt.yticks(rotation=0)
    return g

In [ ]:
plt.figure(figsize=[10,4])
g = crear_grafico_heatmap(_heatmap, 'Blues', 'Eventos registrados por hora, por día')
plt.show()

## Filtrando según cómo accedió el usuario al sitio

Ahora vamos a filtrar los eventos en tres categorías, según cómo accedió el usuario al sitio.

## Visited Site (visita directa a través de un link)

In [ ]:
visitas_directas = eventos_df.loc[eventos_df['event'] == 'visited site', :]
_heatmap = visitas_directas.pivot_table(index='weekday', columns='hour', values='event', aggfunc='count')

In [ ]:
plt.figure(figsize=[10,4])
g = crear_grafico_heatmap(_heatmap, 'Reds', 'Visitas directas registradas por hora, por día')
plt.show()

## Ad Campaign Hit (acceso mediante una publicidad)

In [ ]:
visitas_publicidad = eventos_df.loc[eventos_df['event'] == 'ad campaign hit', :]
_heatmap = visitas_publicidad.pivot_table(index='weekday', columns='hour', values='event', aggfunc='count')

In [ ]:
plt.figure(figsize=[10,4])
g = crear_grafico_heatmap(_heatmap, 'Purples', 'Visitas desde publicidades registradas por hora, por día')
plt.show()

## Search Engine Hit (el usuario accede desde un buscador web)

In [ ]:
visitas_buscadores = eventos_df.loc[eventos_df['event'] == 'search engine hit', :]
_heatmap = visitas_buscadores.pivot_table(index='weekday', columns='hour', values='event', aggfunc='count')

In [ ]:
plt.figure(figsize=[10,4])
g = crear_grafico_heatmap(_heatmap, 'Greens', 'Visitas desde buscadores registradas por hora, por día')
plt.show()

## Observaciones respecto al tráfico según tipo de acceso al sitio

Podemos notar claramente que el comportamiento de los usuarios respecto al acceso al sitio se mantiene independientemente del tipo de evento.

En primer lugar, vemos que hay mayor frecuencia de eventos durante los días de semana que durante los fines de semana.

Además se puede observar que en los tres casos expuestos, hay más accesos al sitio durante el horario laboral, presentando una pequeña disminución entre las 19 y 21 horas, para luego volver a incrementarse la frecuencia hasta las primeras horas de la madrugada. 

La franja horaria comprendida entre las 5 y las 9 de la mañana presenta actividad nula.

De todas formas por falta de datos demográficos sobre los usuarios del sitio, creo prudente no realizar afirmaciones respecto al motivo del comportamiento de los usuarios.

# Productos vistos, checkouts y compras

Procederemos a realizar heatmaps para estos eventos, ya que nos interesa saber si el comportamiento de los usuarios en estos eventos es similar al que observamos en las visitas al sitio.

In [ ]:
checkouts = eventos_df.loc[eventos_df['event'] == 'checkout',['timestamp', 'weekday', 'hour', 'event']]
_heatmap = checkouts.pivot_table(index='weekday', columns='hour', values='event', aggfunc='count')

In [ ]:
plt.figure(figsize=[10,4])
g = crear_grafico_heatmap(_heatmap, 'Oranges', 'Checkouts registrados por hora, por día')
plt.show()

In [ ]:
conversiones = eventos_df.loc[eventos_df['event'] == 'conversion',['timestamp', 'weekday', 'hour', 'event']]
_heatmap = conversiones.pivot_table(index='weekday', columns='hour', values='event', aggfunc='count')

In [ ]:
plt.figure(figsize=[10,4])
g = crear_grafico_heatmap(_heatmap, 'Greys', 'Conversiones registradas por hora, por día')
plt.show()

In [ ]:
productos_vistos = eventos_df.loc[eventos_df['event'] == 'viewed product', ['timestamp', 'weekday', 'hour', 'event']]
_heatmap = productos_vistos.pivot_table(index='weekday', columns='hour', values='event', aggfunc='count')

In [ ]:
plt.figure(figsize=[10,4])
g = crear_grafico_heatmap(_heatmap, 'Blues', 'Productos vistos por hora, por día')
plt.show()